In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

import time
import os
import copy

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
data_transforms = {'training':transforms.Compose([transforms.RandomResizedCrop(224),
                                              transforms.RandomHorizontalFlip(),
                                              transforms.ToTensor(),
                                              transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])]),
                  'test': transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])}

In [17]:
data_dir = 'drive/MyDrive/ai'

In [18]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x),
                                         data_transforms[x])
                   for x in {'training','test'}}
dataloaders = {x:DataLoader(image_datasets[x],
                           batch_size= 4,
                           shuffle=True,
                           ) for x in ['training','test']}
dataset_size = {x:len(image_datasets[x]) for x in['training','test']}
class_names = image_datasets['training'].classes

In [19]:
print(class_names)

['1', '2', '3', '4', '5']


In [20]:
model_ft = models.resnet18(pretrained = True)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]



---



In [21]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [22]:
#데이터의 수에 맞게 out_features 변경
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,5)
model_ft = model_ft.to(device)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(),lr = 0.001,momentum=0.9)


In [24]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,step_size=7,gamma=0.1)

In [30]:
def train_model(model,criterion,optimizer,scheduler,num_epochs = 25): #학습 관련된것 전부 집어 넣음
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict()) # 모델의 상태값 가졍하
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print("Epoch number {}/{}".format(epoch,num_epochs -1))
        print('-' * 10)
        
        for phase in ['training','test']: 
            if phase == 'training': 
                model.train() 
            else:
                model.eval() #평가해줘

            running_loss = 0.0 #로스초기화
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() #옵티마이저 초기화(경사도초기화)

                with torch.set_grad_enabled(phase == 'training'):
                    outputs = model(inputs) #train단계 에서만, 인풋을 넣으면 아웃풋이 나오게
                    _, preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels) #loss로 아웃풋과 레이블 비교

                if phase == 'test':
                    loss.requires_grad_(True)
                    loss.backward() 
                    optimizer.step() #옵티마이저 스텝 증가

                running_loss += loss.item() * inputs.size(0) #로스 아이템에 인풋 사이즈 곱해
                running_corrects += torch.sum(preds == labels.data) #예측과 정답이 같을때

            if phase == 'training':
                scheduler.step() #러닝스케줄러 증가

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double()/ dataset_size[phase]

            print('{} Loss : {:.4f}\tAccuracy: {:4f}'.format(phase,epoch_loss,epoch_acc))
            best_acc = epoch_acc
            if phase == 'test' and epoch_acc > best_acc:

                best_model_wts = copy.deepcopy(model.state_dict())

            print()
        time_elapsed = time.time() - since
        print('Traing time:{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best: {:4f}'.format(best_acc))

        model.load_state_dict(best_model_wts)
    
    return model

In [31]:
model_ft = train_model(model_ft,criterion,optimizer_ft,exp_lr_scheduler,num_epochs=25)

Epoch number 0/24
----------
training Loss : 1.7944	Accuracy: 0.198113



KeyboardInterrupt: ignored

# 시각화


In [ ]:
def imshow(img):
    img = img/2 +0.5 
    npimg = img.numpy()
    fig = plt.figure(figsize=(10,5))
    plt.imshow(np.transpose(npimg,(1,2,0)))
    plt.show()

In [ ]:
def visulaize_model(model, num_images=6): #이미지 갯수 지정한 만큼
    was_traing = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure
    
    with torch.no_grad():
        for i, (inputs,labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            
            for j in range(inputs.size()[0]):
                images_so_far +=1
                ax = plt.subplot(num_images // 2,2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted:{}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])
                
                if images_so_far == num_images:
                    model.train(mode = was_traing)
                    return
                
    model.train(model=was_traing)

In [ ]:
visulaize_model(model_ft)